In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
from datetime import timedelta  
import pyautogui
import ast
from requests_html import HTMLResponse, HTMLSession
from bs4 import BeautifulSoup
import requests

In [20]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)

Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0


In [ ]:
count = 0
list_images = []


note = ''

def extrair_video_url():
    url_video = ''
    click_videos = driver.find_element_by_css_selector('.wistia-popover\[height\=337\,playerColor\=7b796a\,width\=550\]')
    click_videos.click()
    time.sleep(1)
    videos = driver.find_elements_by_xpath('//iframe')
    for i in videos:
        src = i.get_attribute('src')

        if 'http' in src:
            url_video = src

    time.sleep(0.5)
    close_videos = driver.find_element_by_class_name('fancybox-item')
    close_videos.click()
    return url_video


def extrair_img():
    url_image = driver.find_element_by_id('preview')
    print('preview trouver')
    url_image.get_attribute('innerHTML')
    soup = BeautifulSoup(url_image.get_attribute('innerHTML'), 'html.parser')
    src = soup.find('img')
    src = src['src']
    return src


def extraire_function(word, list_word):
    for t in list_word:
        if word in t:
            return t.replace(word, '').replace(':', '')
#**************************************************************************
#**************************************************************************
#**************************************************************************
#**************************************************************************
list_urls = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/tavolashop/tavolashop-cat_toto.xlsx')

urls =  []
for index, row in list_urls.iterrows():
    urls.append((row['url'], row['cat1'], row['cat2'], row['cat3']))
    
name_excel = 'tavolashop-update_ar.xlsx'
df = pd.read_excel('tavola_model.xlsx')
#ForLoop main
for i , product in enumerate(urls[530:]):
    print('*****************************************************************')
    print('Count:', i)
    try:
        driver.get(product[0])
        time.sleep(1)
        sku = 'TAV-' +driver.find_element_by_class_name('value').text
        
    except:
        print('pas de produit')
        continue
    try:
        title = driver.find_element_by_class_name('base').text
        
    except:
        continue
#**************************************************************************


    link_url = str(product[0])
    try:
        in_stock = driver.find_element_by_class_name('stock').text
    except:
        in_stock = driver.find_element_by_class_name('available').text

#**************************************************************************
    try:
        mgs_brand = driver.find_element_by_class_name('brand-link').get_attribute('title')
    except:
        mgs_brand = ''
#**************************************************************************
    try:
        prices = driver.find_element_by_class_name('price-final_price')
    except:
        continue
    try:
        price = prices.find_element_by_class_name('old-price').text
        special_price = prices.find_element_by_class_name('price').text
    except:
        try:
            price = prices.find_element_by_class_name('price').text
            special_price = '0'
        except:
            continue
    print('Price: ', price)
    print('S Price: ', special_price)

#**************************************************************************
    try:
        estimated_shipping = driver.find_element_by_class_name('product_detail_estimated_shipping').text.split(':')[1]
    except:
        estimated_shipping = '__EMPTY__VALUE__'
#**************************************************************************
        
    try:
        products_marque = driver.find_element_by_class_name('brand-link').get_attribute('href')
    except:
        products_marque = ''
    try:
        
        image_marque = driver.find_element_by_class_name('brand-image').get_attribute('src')
    except:
        image_marque = ''
#**************************************************************************
    
#     except:
#         print('continue')
#         continue

    
 # CATEGORY  *******************************************   
 
    try:
        category1 =  product[1]
    except:
        category1 = ''
    try:
        category2 = product[2]
    except:
        category2 = ''
    try:
        category3 = product[3]
    except:
        category3 = ''
    print('Cat1:', category1)
    print('Cat2:', category2)
    print('Cat3:', category3)
    
   # Click and extract discription  *******************************************    
    try:
        btn_additional = driver.find_element_by_id('tab-label-additional')
        driver.execute_script("arguments[0].click();", btn_additional)
    except:
        pass
    try:
        additinal = driver.find_element_by_id('additional')
        list_additional = additinal.text.split('\n')
    except:
        list_additional = ''
 # Size  *******************************************   


 # Color  *******************************************   
    color = extraire_function('اللون', list_additional)
    if color == '':
        color = extraire_function('الللون', list_additional)
    if color == '':
        r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
        soup = BeautifulSoup(r, 'html.parser')
        try:
            color = soup.find('th', text=re.compile('اللون')).next_element.next_element.text.strip()
        except:
            color = ''
    if color == '' or color == None:
        color = '__EMPTY__VALUE__'
    print('Color:', color)
    
 # Size  *******************************************   
    size = extraire_function('الأبعاد', list_additional)
    if size == '':
        size = extraire_function('القياسات', list_additional)
    if size == '':
        size = '__EMPTY__VALUE__'
    print('Size:', size)
# Matter ***************************************************
    matter = extraire_function('المادة', list_additional)
    if matter == '':
        r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
        soup = BeautifulSoup(r, 'html.parser')
        try:
            matter = soup.find('th', text=re.compile('المادة الأساسية')).next_element.next_element.text.strip()
        except:
            matter = ''
    if matter == '':
        matter = '__EMPTY__VALUE__'
    print('Matter:', matter)
    time.sleep(2)
# IMAGES ***************************************************
    list_images = []
    images = driver.find_elements_by_class_name('fotorama__nav__frame--thumb')

    if images == []:
        image = driver.find_element_by_class_name('fotorama__img').get_attribute('src')
        time.sleep(1)
        list_images.append(image)
    else:
        len(images)
        for i in images:
            try:
                
                i.click()
                time.sleep(1)

                list_images.append(extrair_img())


            except:
                print('Pas d images')

    print('Images:', list_images)

    # Videos ******************************************************
    
    try :
        click_videos = driver.find_elements_by_xpath("//a[contains(@class, 'wistia-popover')]")
        print('Click video', len(click_videos))
        if len(click_videos) != 0:
            click_videos[0].click()
            print('je suis dans le 2')
            pip = driver.find_elements_by_xpath('//iframe')
            len(pip)
            regex = re.compile('^https:*')
            for p in pip:
                if regex.search(str(p.get_attribute('src'))) != None:
                    url_video = p.get_attribute('src')
        else:
            url_video = ''
    except:
        url_video = ''
    
    
    if url_video == '':
        videos = driver.find_elements_by_xpath('//*[contains(text(),"Video")]')
        print(len(videos))
        if len(videos) != 0:
            print('Video extract')
            toto = videos[0].get_attribute('innerHTML')
            dic = toto.replace('[', '').replace(']', '').replace("'", '')
            toto_dic = ast.literal_eval(dic)
            url_video = toto_dic['@id']
        else:
            url_video = ''
    print('URL video:', url_video)



    # Specifications *******************************************
    try:
        try:
            btn_additional = driver.find_element_by_id('tab-label-additional').click()
        except:
            btn_additional = driver.find_element_by_id('tab-label-additional')
            driver.execute_script("arguments[0].click();", btn_additional)
            time.sleep(0.5)
    except:
        continue
    try:
        specifications = driver.find_element_by_xpath('//td[@data-th="Specification"]').text

    except:
        specifications = ''
    
    #  ***************************************************
    
    try:
        description = driver.find_element_by_class_name('description').text
    except:
        description = ''
    
    # *********************************************************
    try:
        overview = driver.find_element_by_class_name('overview').text.split('\n')[0]

    except:
        overview = ''
    
    try:
        set_include = "".join(driver.find_element_by_class_name('overview').text.split('\n')[1:])
    except:
        set_include = " " 
    # ****************************************************
    url_key = f'{sku}-{title}'
    
    url_key = url_key.replace(',', '-').replace('*', '-').replace('،', '-')
    
 #  ***************************************************

    two_month = datetime.now() + timedelta(days=60)
    two_month = two_month.strftime("%m/%d/%Y")
    today = datetime.today().strftime("%m/%d/%Y")
 #  ***************************************************

    if in_stock == 0:
        product_online = 2
    else:
        product_online = in_stock
    
#     list_input_width.append(width)
#     list_input_height.append(height)
#     list_input_length.append(length)
#     list_input_note.append(note)
#     list_input_product_size.append(size)


    try:
        base_image = list_images[0]
    except:
        base_image = ''
    small_image = base_image
    swatch_image = base_image
    thumbnail_image = base_image
    additional_images = ','.join(list_images[1:])
    
#     try:
#         list_input_images3.append(list_images[1])
#     except:
#         list_input_images3.append('')
#     try:
#         list_input_images4.append(list_images[3])
#     except:
#         list_input_images4.append('')
    #df['Images'] = list_images
    series = pd.Series([sku,
                        in_stock,
                        estimated_shipping,
                     title,
                     url_key,

                     color,
                     matter,
                    overview,
                    specifications,
                    '__EMPTY__VALUE__',#width,
                    '__EMPTY__VALUE__',#height,
                    '__EMPTY__VALUE__',#length,
                    '__EMPTY__VALUE__',#weight,
                     base_image,
                     small_image,
                     swatch_image,
                     thumbnail_image,
                    mgs_brand,
                     additional_images ,
                     category1,
                     category2,
                     category3,
                     price ,
                    special_price,
                    'Catalog, Search',
                    'Taxable Goods',
                    link_url,
                    0,
                    -5,
                    1,
                    1,
                    1,
                    'مستوردة',
                    url_video,
                     today,
                     two_month,
                     'base',
                     'Default',
                     'simple',
                     '',
                    'TAV'

                    ],  index=df.columns)
    #df.append(df1)
    df = df.append(series, ignore_index=True)
    df.to_excel( f'{name_excel}', index = False)


In [29]:
url = pd.read_excel('product.template.xlsx')

In [30]:
urls = url['default_code']

In [28]:
df.to_excel('TAvola.update.xlsx')

In [31]:
df1 = df.loc[df['sku'].isin(urls)]

In [18]:
df1.to_excel('tavolashop-update_productsfind.xlsx')